# CNN Classifier - Training

<hr>

This notebook contains a script for training a CNN classifier implemented in cnn_clf.py

Classifier is implemented in PyTorch and utilizes the structure of PyTorch Lighting which also enables to train our model on TPUs.

## **Setup**

- install modules necessary for TPU computation
- install torch==1.5.0 and pytorch-lightnint
- import libraries and mount Google Drive

In [1]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4264  100  4264    0     0  23048      0 --:--:-- --:--:-- --:--:-- 23048
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
Uninstalling torch-1.5.0:
  Successfully uninstalled torch-1.5.0
Uninstalling torchvision-0.7.0a0+34810c0:
  Successfully uninstalled torchvision-0.7.0a0+34810c0
Copying gs://tpu-pytorch/wheels/torch-nightly-cp36-cp36m-linux_x86_64.whl...
\ [1 files][ 91.9 MiB/ 91.9 MiB]                                                
Operation completed over 1 objects/91.9 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly-cp36-cp36m-linux_x86_64.whl...
\ [1 files][122.2 MiB/122.2 MiB]                                                
Operation completed over 1 objects/122.2 MiB.                                    
Copying gs://tpu-pyt

In [2]:
# insall pytorch, pytorch-ligthning
!pip install torch==1.5.0
!pip install pytorch-lightning

# import basic data science lib
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

# mport torch-related modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer
from torch.utils.data import DataLoader, TensorDataset

# mount a google drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
%cd 'drive/My Drive'

time.sleep(5)
from cnn_clf import * # the best practive would be store a script with the CNN class on github and clone the repo

  Using cached https://files.pythonhosted.org/packages/13/70/54e9fb010fe1547bc4774716f11ececb81ae5b306c05f090f4461ee13205/torch-1.5.0-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: torch 1.6.0a0+68f23d5
    Uninstalling torch-1.6.0a0+68f23d5:
      Successfully uninstalled torch-1.6.0a0+68f23d5
Mounted at /content/drive
/content/drive/My Drive


## **Load the data**

In [0]:
label = 'label_top5'

In [0]:
# helper loading function
def X_loader(dataset):
  return torch.Tensor(
      np.load(f'SeznamResearch/splitted_data/cnn/{dataset}.npy')
  )
def y_loader(dataset):
  return torch.Tensor(
      pd.read_csv(f'SeznamResearch/splitted_data/bert/{dataset}.csv')[label]
  )

# load feature matrices
X_train = X_loader('train')
X_dev = X_loader('dev')
X_test = X_loader('test')

# load labels
y_train = y_loader('train')
y_dev = y_loader('dev')
y_test = y_loader('test')

# complete tuple (X, y)
datasets = {
    'train': TensorDataset(X_train, y_train),
    'val': TensorDataset(X_dev, y_dev),
    'test': TensorDataset(X_test, y_test)
}

## **Run training**

In [0]:
### editable parameters
dropout_set = [0.1, 0.2, 0.3, 0.5]
kernel_num_set = [64, 128, 256, 512]

# set early stopping rule
early_stop_callback = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0,
    patience=5,
    verbose=1,
    mode='min'
)

############# don't edit #############
val_f1 = np.zeros(
    (len(dropout_set), len(kernel_num_set))
)
test_f1 = np.zeros_like(val_f1)

# run training
for i,dropout in enumerate(dropout_set):
  for j,kernel_num in enumerate(kernel_num_set):
    # set parameters and instantiate net
    cnn_parameters = {
        'batch_size': 32,
        'kernel_num': kernel_num,
        'kernel_filters': (3,4,5),
        'embed_dim': 100,
        'learning_rate': 4e-3,
        'dropout': dropout,
        'num_classes': 6,
        'datasets': datasets
    }
    cnn = CNN_classifier(**cnn_parameters)

    # define trainer and run training
    try:
      del trainer
    except:
      pass
    trainer = Trainer(
      tpu_cores=8,
      check_val_every_n_epochs=2,
      early_stop_callback=early_stop_callback,
      max_epochs=50,
      progress_bar_refresh_rate=0
    )
    trainer.fit(cnn)
    
    # store f1-score on val and test set
    val_f1[i,j] = f1_score(
        y_dev.detach().numpy(), # y_true
        cnn(X_dev).argmax(1).detach().numpy(), # y_pred
        average='macro'
    )
    print(
        f'Kernel num = {kernel_num}, Dropout = {dropout}: F1-macro = {val_f1[i,j]:.4f}'
    )
    test_f1[i,j] = f1_score(
        y_test.detach().numpy(), # y_true
        cnn(X_test).argmax(1).detach().numpy(), # y_pred
        average='macro'
    )

# save the results
np.save('val_f1.npy', val_f1) # the best classifier is selected based on val_score; all test score are generated for the purpose of simplicity of sampling later on
np.save('test_f1.npy', test_f1)

GPU available: False, used: False
No environment variable for node rank defined. Set as 0.

  | Name              | Type       | Params
---------------------------------------------
0 | conv_layers       | ModuleDict | 76 K  
1 | conv_layers.conv1 | Conv2d     | 19 K  
2 | conv_layers.conv2 | Conv2d     | 25 K  
3 | conv_layers.conv3 | Conv2d     | 32 K  
4 | fc                | Linear     | 1 K   
5 | dropout           | Dropout    | 0     
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the 

Epoch 1 - Validation loss = 1.7634
Epoch 2 - Validation loss = 0.4010


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Did not find hyperparameters at model hparams. Saving checkpoint without hyperparameters.
  warnings.warn(*args, **kwargs)


Epoch 3 - Validation loss = 0.3571
Epoch 4 - Validation loss = 0.3397
Epoch 5 - Validation loss = 0.3336
Epoch 6 - Validation loss = 0.3385


Epoch 00006: early stopping
GPU available: False, used: False
No environment variable for node rank defined. Set as 0.

  | Name              | Type       | Params
---------------------------------------------
0 | conv_layers       | ModuleDict | 153 K 
1 | conv_layers.conv1 | Conv2d     | 38 K  
2 | conv_layers.conv2 | Conv2d     | 51 K  
3 | conv_layers.conv3 | Conv2d     | 64 K  
4 | fc                | Linear     | 2 K   
5 | dropout           | Dropout    | 0     


...

Epoch 12 - Validation loss = 0.3306
Kernel num = 256, Dropout = 0.5: F1-macro = 0.8388


GPU available: False, used: False
No environment variable for node rank defined. Set as 0.

  | Name              | Type       | Params
---------------------------------------------
0 | conv_layers       | ModuleDict | 615 K 
1 | conv_layers.conv1 | Conv2d     | 154 K 
2 | conv_layers.conv2 | Conv2d     | 205 K 
3 | conv_layers.conv3 | Conv2d     | 256 K 
4 | fc                | Linear     | 9 K   
5 | dropout           | Dropout    | 0     


Epoch 1 - Validation loss = 1.8034
Epoch 2 - Validation loss = 0.4179
Epoch 3 - Validation loss = 0.3699
Epoch 4 - Validation loss = 0.3519
Epoch 5 - Validation loss = 0.3389
Epoch 6 - Validation loss = 0.3238
Epoch 7 - Validation loss = 0.3371
Epoch 8 - Validation loss = 0.3108
Epoch 9 - Validation loss = 0.3260


Epoch 00009: early stopping


Epoch 10 - Validation loss = 0.3257
Kernel num = 512, Dropout = 0.5: F1-macro = 0.8236
